In [3]:
import pandas as pd
import numpy as np
import os
import pickle
import seaborn as sns 

In [4]:
df_new = pd.read_csv('movie_reviews.csv')

In [5]:
df_new.head()

,review,sentiment
0,This is one of those unfortunate films that su...,1
1,Okay maybe it was because I happen to be in Ya...,1
2,"Although I love this movie, I can barely watch...",1
3,"A man arrives in a strange, beautiful, sterile...",1
4,I'm sitting around going through movie listing...,1


In [6]:
df_new.sentiment.value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [7]:
df_new.columns

Index(['review', 'sentiment'], dtype='object')

In [8]:
df_new.dtypes

review       object
sentiment     int64
dtype: object

In [9]:
df_new.isnull().isnull().sum()

review       0
sentiment    0
dtype: int64

In [10]:
df_new.loc[1000,'review']

"I rented this movie last week. I saw Kevin Spacey and Morgan Freeman were on it, so it seemed promising. And it was, until Justin Timberlake came on scene. He is a really bad actor and shouldn't be allowed to make a movie ever again. I mean, he is one of the most boring, uninspired actors I've ever seen. He puts absolutely no emotion to any of his lines whatsoever. Why the hell was he cast for the role of Josh Pollack? I think Matt Damon would have been a better choice.<br /><br />Kevin Spacey was another big disappointment. His character is so dull, it seems like a bad mix of his character in American Beauty and John Doe in Se7en. It might sound cool, but believe me, it's not.<br /><br />Now, Dylan McDermott's acting is very good. It's about one of the very few good things about this movie. He is just inspired.<br /><br />Morgan Freeman is good but nothing special. He has some really cool lines though.<br /><br />About the story, although it was a bit obvious and exaggerated at times

In [11]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    cleanString = re.sub('\W+',' ', text )
    return  cleanString


In [12]:
preprocessor("I rented this movie last week. I saw Kevin Spacey and Morgan Freeman were on it, so it seemed promising. And it was, until Justin Timberlake came on scene. He is a really bad actor and shouldn't be allowed to make a movie ever again. I mean, he is one of the most boring, uninspired actors I've ever seen. He puts absolutely no emotion to any of his lines whatsoever. Why the hell was he cast for the role of Josh Pollack? I think Matt Damon would have been a better choice.<br /><br />Kevin Spacey was another big disappointment. His character is so dull, it seems like a bad mix of his character in American Beauty and John Doe in Se7en. It might sound cool, but believe me, it's not.<br /><br />Now, Dylan McDermott's acting is very good. It's about one of the very few good things about this movie. He is just inspired.<br /><br />Morgan Freeman is good but nothing special. He has some really cool lines though.<br /><br />About the story, although it was a bit obvious and exaggerated at times it was good. I was expecting a big twist when Lazerov (Dylan McDermott) was killed, but nothing really happened.")

'I rented this movie last week I saw Kevin Spacey and Morgan Freeman were on it so it seemed promising And it was until Justin Timberlake came on scene He is a really bad actor and shouldn t be allowed to make a movie ever again I mean he is one of the most boring uninspired actors I ve ever seen He puts absolutely no emotion to any of his lines whatsoever Why the hell was he cast for the role of Josh Pollack I think Matt Damon would have been a better choice Kevin Spacey was another big disappointment His character is so dull it seems like a bad mix of his character in American Beauty and John Doe in Se7en It might sound cool but believe me it s not Now Dylan McDermott s acting is very good It s about one of the very few good things about this movie He is just inspired Morgan Freeman is good but nothing special He has some really cool lines though About the story although it was a bit obvious and exaggerated at times it was good I was expecting a big twist when Lazerov Dylan McDermott

In [13]:
df_new['review'] = df_new['review'].apply(preprocessor)

In [14]:
df_new_data = df_new[["review","sentiment"]]

In [15]:
df_new_data.columns

Index(['review', 'sentiment'], dtype='object')

In [16]:
df_new_x = df_new_data["review"]
df_new_y = df_new_data["sentiment"]


In [17]:
X_train = df_new.loc[:2500, 'review'].values
y_train = df_new.loc[:2500, 'sentiment'].values
X_test = df_new.loc[2500:5000, 'review'].values
y_test = df_new.loc[2500:5000, 'sentiment'].values

In [18]:
#print (np.bincount(y_test))
#print (np.unique(y_test))

#print (np.bincount(y_train))

In [19]:
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

In [20]:
tfidf = TfidfVectorizer(stop_words='english')

param_grid = {'clf__C': [1.0, 10.0, 100.0]}

In [21]:
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression())])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',)


In [22]:
lr_tfidf.steps

[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
          stop_words='english', strip_accents=None, sublinear_tf=False,
          token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None)),
 ('clf',
  LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l2', random_state=None, solver='warn',
            tol=0.0001, verbose=0, warm_start=False))]

In [23]:
gs_lr_tfidf.fit(X_train, y_train)

C:\Users\rohit\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\rohit\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__C': [1.0, 10.0, 100.0]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [24]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 10.0} 
CV Accuracy: 0.837


In [25]:
clf = gs_lr_tfidf.best_estimator_

In [26]:
clf.score(X_test,y_test)

0.8548580567772891

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [28]:
nb = Pipeline([('vect', tfidf),
               ('clf', MultinomialNB())])

In [29]:
nb.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [30]:
nb.score(X_train,y_train)

0.9540183926429429

In [31]:
nb.score(X_test,y_test)

0.7896841263494602

In [32]:
cv = CountVectorizer(stop_words='english')

In [33]:
new_data = cv.fit_transform(X_train)
new_test = cv.transform(X_test)

In [34]:
new_data.shape

(2501, 28031)

In [35]:
nb = MultinomialNB()
nb.fit(new_data,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [36]:
nb.score(new_test,y_test)

0.8208716513394642

In [37]:
nb.score(new_data,y_train)

0.9684126349460216

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
cv = CountVectorizer(stop_words = 'english')
ex = cv.fit_transform(["Okay maybe it was because I happen to be in Yangchun China when I saw this movie. Maybe it was because I finally had something on TV I could understand or at least read the subtitles, or maybe it was just funny. Whatever it was this movie was worth the time.<br /><br />I had just arrived for my foot and head massage when they gave me the remote so I could watch TV. Usually I would turn the darn thing off but I stumbled upon this crazy movie and got hooked.<br /><br />The plot if you could call it a plot sort of revolves around a cooking competition and sort of is a love story and the food in this movie is the real star. If you like Iron Chef and many of the other cooking shows currently in the reality TV mode, then you will love the scenes with food in this movie.<br /><br />It goes fast and the subtitles are so fast you better be up on speed reading for this one. However the action is mostly slapstick so you don't always have to read the entire subtitle to get the idea.<br /><br />The main actress is lovely eye candy and the main actor isn't bad to look at either. They are both worth watching. Finally if you have some time to kill and want a good laugh this isn't a bad choice for both.<br /><br />I don't speak a word of Chinese but I was totally able to understand the cultural humor of this film. For those who do speak Chinese maybe it is even better. Overall I give this an 8 out of 10 and currently I am even looking to find a copy to have while I stay in China, and keep for when I come back home, it will be a nice reminder for me of my time in Yangchun and a silly afternoon at the massage salon watching a silly movie."])

In [40]:
ex.toarray()

array([[ 1,  1,  1,  1,  1,  1,  1,  2,  2, 10,  1,  1,  2,  2,  1,  1,
         1,  2,  1,  1,  1,  2,  1,  2,  1,  1,  2,  1,  2,  2,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  1,  1,  1,
         1,  1,  2,  1,  2,  2,  4,  1,  6,  1,  1,  1,  2,  2,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  2,  2,  1,  1,  1,  1,
         1,  1,  2,  1,  3,  1,  1,  3,  2,  1,  1,  1,  2,  1,  2,  2]],
      dtype=int64)

In [41]:
cv.get_feature_names()

['10',
 'able',
 'action',
 'actor',
 'actress',
 'afternoon',
 'arrived',
 'bad',
 'better',
 'br',
 'candy',
 'chef',
 'china',
 'chinese',
 'choice',
 'come',
 'competition',
 'cooking',
 'copy',
 'crazy',
 'cultural',
 'currently',
 'darn',
 'don',
 'entire',
 'eye',
 'fast',
 'film',
 'finally',
 'food',
 'foot',
 'funny',
 'gave',
 'goes',
 'good',
 'got',
 'happen',
 'head',
 'home',
 'hooked',
 'humor',
 'idea',
 'iron',
 'isn',
 'just',
 'kill',
 'laugh',
 'like',
 'look',
 'looking',
 'love',
 'lovely',
 'main',
 'massage',
 'maybe',
 'mode',
 'movie',
 'nice',
 'okay',
 'overall',
 'plot',
 'read',
 'reading',
 'real',
 'reality',
 'reminder',
 'remote',
 'revolves',
 'salon',
 'saw',
 'scenes',
 'shows',
 'silly',
 'slapstick',
 'sort',
 'speak',
 'speed',
 'star',
 'stay',
 'story',
 'stumbled',
 'subtitle',
 'subtitles',
 'thing',
 'time',
 'totally',
 'turn',
 'tv',
 'understand',
 'usually',
 'want',
 'watch',
 'watching',
 'word',
 'worth',
 'yangchun']

In [42]:
corpus = df_new_x
cv = CountVectorizer(stop_words = 'english')
X = cv.fit_transform(corpus)

In [43]:
cv.get_feature_names()

['00',
 '000',
 '00000000000',
 '0000000000001',
 '00000001',
 '00001',
 '00015',
 '000dm',
 '000s',
 '001',
 '003830',
 '006',
 '0069',
 '007',
 '0079',
 '007s',
 '0080',
 '0083',
 '009',
 '0093638',
 '00am',
 '00o',
 '00pm',
 '00s',
 '00schneider',
 '01',
 '0126',
 '0148',
 '01pm',
 '02',
 '020410',
 '0230',
 '029',
 '02i',
 '02year',
 '03',
 '039',
 '04',
 '041',
 '044',
 '05',
 '050',
 '05nomactr',
 '06',
 '0615',
 '06th',
 '07',
 '07b',
 '08',
 '087',
 '089',
 '08th',
 '09',
 '0and',
 '0f',
 '0ne',
 '0r',
 '0s',
 '0ttmay',
 '10',
 '100',
 '1000',
 '10000',
 '1000000',
 '10000000000',
 '10000000000000',
 '10000th',
 '1000lb',
 '1000s',
 '1000th',
 '1001',
 '1004',
 '100b',
 '100bt',
 '100ft',
 '100ibs',
 '100k',
 '100m',
 '100miles',
 '100min',
 '100mins',
 '100mph',
 '100s',
 '100th',
 '100times',
 '100x',
 '100yards',
 '101',
 '101st',
 '102',
 '102nd',
 '103',
 '104',
 '1040',
 '1040a',
 '1040s',
 '105',
 '1050',
 '105lbs',
 '106',
 '106min',
 '107',
 '108',
 '1080',
 '1080p',
 

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test =  train_test_split(X,df_new_y,test_size=0.33, random_state=42)

In [46]:
X_train

<33500x103771 sparse matrix of type '<class 'numpy.int64'>'
	with 2952158 stored elements in Compressed Sparse Row format>

In [47]:
from sklearn.naive_bayes import MultinomialNB

In [48]:
nb = MultinomialNB()
nb.fit(X_train,y_train)
nb.score(X_test,y_test)

0.8566666666666667

In [49]:
print("accuracy of the model",nb.score(X_test,y_test)*100,"%")

accuracy of the model 85.66666666666667 %


In [50]:
nb.predict(X_test)

array([0, 1, 1, ..., 1, 1, 1], dtype=int64)

In [51]:
review = ["Awesome storyline and fantastic acting by every actor. The shown issues are very common in our society which are easily get listened in families and friends."]
vect = cv.transform(review).toarray()

In [52]:
nb.predict(vect)

array([1], dtype=int64)

In [53]:
sentiment_dict = {'negative':0,'positive':1}


In [54]:
sentiment_dict.values()

dict_values([0, 1])

In [55]:
if nb.predict(vect) == 1:
    print("positive")
else:
        print("negative")


positive


In [56]:
import pickle

In [57]:
from sklearn.externals import joblib

In [60]:
Navie_bayes = open("sent_model.pkl","wb")

In [61]:
pickle.dump(nb,Navie_bayes)

In [62]:
Navie_bayes.close()

In [63]:
sent_model = open("sent_model.pkl","rb")

In [64]:
new_model = pickle.load(sent_model)

In [68]:
joblib.dump(nb,'sent_joblib')

['sent_joblib']

In [69]:
new = joblib.load('sent_joblib')

In [65]:
model = MultinomialNB()
model.fit(new_data,X_train)

ValueError: bad input shape (33500, 103771)

In [39]:
i=1
while (i==1):
    rev=input("enter the review")
    preprocessor(rev)
    sentiment=nb.predict(tfidf.transform([rev]))[0]
    if sentiment==1:
        print("good")
    else :
        print("bad")
    i=int(input("do u want to try with more reviews \n")) 
    if i!=1:
        print(" \n thanks for using our project")

enter the reviewmovie is very good
bad
do u want to try with more reviews 
0
 
 thanks for using our project


In [33]:
#pickle.dump(nb,open('model.pkl','wb'))

In [40]:
import pickle

In [41]:
Navie_bayes = open("sent_model.pkl","wb")

In [43]:
pickle.dump(nb,Navie_bayes)

In [45]:
Navie_bayes.close()

In [46]:
sent_model = open("sent_model.pkl","rb")

In [47]:
new_model = pickle.load(sent_model)

In [34]:
#model=pickle.load(open('model.pkl','rb'))

In [35]:
#pickle.dump(tfidf,open('tfidf.pkl','wb'))

In [36]:
#tfidf=pickle.load(open('tfidf.pkl','rb'))

In [37]:
#pickle.dump(cv,open('cv.pkl','wb'))

In [38]:
#cv=pickle.load(open('cv.pkl','rb'))

In [39]:
#joblib.dump(tfidf,"tfidf_joblib") 

In [ ]:
#loaded_tfidf = joblib.load("tfidf_joblib")

In [ ]:
#joblib.dump(model,"model_joblib")

In [ ]:
loaded_model = joblib.load("model_joblib")

In [ ]:
joblib.dump(cv,"cv_joblib")

In [ ]:
loaded_cv = joblib.load("cv_joblib")

In [ ]:
#naive_bayesNB = open("sent analysis_model.pkl","wb")
#pickle.dump(clf,naive_bayesNB)
#naive_bayesNB.close()

load the model

In [ ]:
#sent_model =open("sent analysis_model.pkl","rb")

In [ ]:
#new_model = pickle.load(sent_model)